In [1]:
import pandas as pd
from pandas import ExcelWriter
import re
import concurrent.futures
import datetime
import goslate
import os 
import pyodbc 
from datetime import timedelta

In [2]:
hoy = datetime.date.today()

In [3]:
def crear_df(dia):
    
    fecha = datetime.date.today()
    #PORQUE CORRIÓ AYER,
    fecha = fecha - timedelta(days = dia)
    outputs = os.listdir('../Salida')
    outputs = [i for i in outputs if i.find(str(fecha)) != -1]
    salidas =  ['dfShein_lenceria','dfShein_jeans','dfShein_Blusas','dfShein_dress','dfShein_sweater','dfShein_remeras','dfShein_plus']

    outputs = [i for i in outputs if i[:i.find('2')] in salidas]

    outputs = ['../Salida/'+i for i in outputs]
    outputs = list(map(pd.read_excel,outputs))

    df = pd.DataFrame([])
    for chunk in outputs:
        df = df.append(chunk,sort=False)
    return df

In [4]:
# ctrl + } all comments

In [5]:
#EL PARAMETRO SON LOS DIAS PASADOS EN LOS QUE CORRIÓ SHEIN ( 1 = AYER)
dfShein = crear_df(1)

In [6]:
#Lenceria,
#Jeans,
#Blusas,
#Dress,
#Sweaters,
#Remeras,
#Plus
dfShein['Fecha'].unique()

array(['2020/03/16'], dtype=object)

In [7]:
#Con esto veo los que son NAN
dfShein[dfShein.isna().any(axis=1)]

,pos,Tipo,Color,id Producto,Descripcion,Talle,Stock,Precio,Precio_dto,Url Imagen,Url Producto,pagina_scraper,Fecha,Marca,Moneda,Sexo,Descripcion aux


In [8]:
#con esto veo los duplicados
dfShein[dfShein.duplicated(keep=False)]

,pos,Tipo,Color,id Producto,Descripcion,Talle,Stock,Precio,Precio_dto,Url Imagen,Url Producto,pagina_scraper,Fecha,Marca,Moneda,Sexo,Descripcion aux


In [9]:
len(dfShein) == len(dfShein.drop_duplicates())
#== len(dfShein.dropna())

True

In [10]:
len(dfShein.drop_duplicates())

137372

In [11]:
if len(dfShein) == len(dfShein.dropna()) == len(dfShein.drop_duplicates()):
    pass
else:
    print('REVISAR LOS DUPLICADOS o NAN VALUES')
    raise

In [12]:
dfShein['Fecha'].unique()
#dfShein['Fecha'] = '2020/01/30'

array(['2020/03/16'], dtype=object)

In [87]:
#dfShein_ropa_interior_hombre.rename(columns={'scrapeado':'pagina_scraper'},inplace=True)

In [13]:
dfShein.dropna(inplace=True)

In [14]:
dfShein.drop_duplicates(inplace=True)

In [15]:
dfShein.drop_duplicates(['id Producto','Talle','Color','Descripcion','Tipo'],inplace=True)

In [16]:
dfShein.drop_duplicates(['id Producto','Talle','Color','Descripcion','Tipo','Sexo'],inplace=True)

In [17]:
dfShein.drop_duplicates(['id Producto','Talle','Color'],inplace=True)

In [18]:
dfShein.reset_index(drop=True,inplace=True)

In [19]:
len(dfShein)
#137262
#165216
#143998
#180388
#202306
#205085
#209249
#212640
#234301
#214237
#239764
#280064
#234781

137262

In [20]:
dfShein[dfShein[['id Producto','Talle','Color']].duplicated(keep=False)]
#11066 'id Producto','Talle','Color','Descripcion'

,pos,Tipo,Color,id Producto,Descripcion,Talle,Stock,Precio,Precio_dto,Url Imagen,Url Producto,pagina_scraper,Fecha,Marca,Moneda,Sexo,Descripcion aux


In [21]:
dfShein.reset_index(drop=True,inplace=True)

In [22]:
dfShein.dropna(inplace=True)

In [24]:
'''
select count(*), id_sc2_producto,id_sc2_corrida,talle,tipo,color  
from sc2_detalle where flag = 1
group by id_sc2_producto,id_sc2_corrida,talle,tipo,color  
having count(*) >1
'''

'\nselect count(*), id_sc2_producto,id_sc2_corrida,talle,tipo,color  \nfrom sc2_detalle where flag = 1\ngroup by id_sc2_producto,id_sc2_corrida,talle,tipo,color  \nhaving count(*) >1\n'

In [25]:
#dfShein["Precio Original"] = [precio.split('$')[2] if len(precio.split('$'))==3 else precio.split('$')[1] for precio in dfShein["Precio"]]
#dfShein["Precio Original"] = [precio.split()[0] for precio in dfShein["Precio Original"]]
#dfShein["Precio"] = [precio.split('$')[1] for precio in dfShein["Precio"]]

#dfShein["Precio Original"] = (dfShein["Precio Original"]
#                              .str.extract(r"([\d,\.]+)", expand=False)
#                              .astype(float))

#dfShein["Precio"] = (dfShein["Precio"]
#                     .str.extract(r"([\d,\.]+)", expand=False)
#                     .astype(float))

#dfShein["Precio Total"] = (dfShein["Precio Total"].str.extract(r"([\d,\.]+)",expand=False).str.replace(",","").astype(float))

In [23]:
dfShein['tipo_vigente'] = dfShein['Tipo'].copy()

In [24]:
#fecha = datetime.date.today()
writer = ExcelWriter('../Salida/dfSheinFULL'+str(hoy)+'.xlsx')
dfShein.to_excel(writer,'Hoja1')
writer.save()

In [25]:
dfShein['Tipo'].unique()

array(['CHAQUETA', 'SWEATER', 'CARDIGAN', 'BUZO', 'BLAZER', 'VESTIDO',
       'REMERA', 'CAMISETAS S/MANGA', 'ENTERITO', 'JEAN',
       'CHAQUETA DE JEAN', 'SHORT DE JEAN', 'CALZA', 'TOP DEPORTIVO',
       'FALDA', 'SHORT', 'PANTALON', 'PLUS SIZE', 'BLUSA', 'TOP',
       'LENCERIA', 'PIJAMA', 'ROPA DE PLAYA'], dtype=object)

In [ ]:
array(['BLUSA', 'TOP', 'JEAN', 'CHAQUETA DE JEAN', 'SHORT DE JEAN',
       'CALZA', 'TOP DEPORTIVO', 'FALDA', 'SHORT', 'PANTALON', 'PIJAMA',
       'ENTERITO', 'ROPA DE PLAYA', 'PLUS SIZE', 'VESTIDO', 'REMERA',
       'CAMISETAS S/MANGA', 'SWEATER', 'CARDIGAN', 'BUZO', 'BLAZER'],
      dtype=object)

In [22]:
#executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
#gs = goslate.Goslate(executor=executor)
#dfShein["Descripcion aux"] = pd.Series(gs.translate(dfShein["Descripcion"], "es"))

In [26]:
server = 'tcp:192.168.1.6'
database ="planeamiento"
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [27]:
#VER BIEN CUANDO EL ORIGEN ES SHEIN, PORQUE AHORA HAY MÁS SCRAPERS EN LA MISMA BASE
maxCorrida = "select max(id_sc2_corrida)+1 from sc2_detalle where origen ='SHEIN' "
maxCorrida = pd.read_sql(maxCorrida,cnxn)

In [28]:
maxCorrida

,
0,44


In [29]:
dfShein['origen'] = 'SHEIN'
#1 OK
dfShein['flag'] = 1

In [30]:
dfShein.columns

Index(['pos', 'Tipo', 'Color', 'id Producto', 'Descripcion', 'Talle', 'Stock',
       'Precio', 'Precio_dto', 'Url Imagen', 'Url Producto', 'pagina_scraper',
       'Fecha', 'Marca', 'Moneda', 'Sexo', 'Descripcion aux', 'tipo_vigente',
       'origen', 'flag'],
      dtype='object')

In [30]:
cursor = cnxn.cursor()
for index,row in dfShein.iterrows():
    cursor.execute("INSERT INTO sc2_detalle([id_sc2_producto], [id_sc2_corrida],[talle],[marca],[tipo],[color],[sexo],[descripcion_aux],[descripcion],[stock],[moneda],[precio],[precio_original],[fecha_alta],[url_imagen],[url_producto],[origen],[tipo_vigente],[posicion],[pagina],[flag])values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                   row['id Producto'], 
                   int(maxCorrida[""][0]),
                   row['Talle'],
                   row['Marca'],
                   row['Tipo'],
                   row['Color'],
                   row['Sexo'],
                   row['Descripcion aux'],
                   row['Descripcion'], 
                   row['Stock'],
                   row['Moneda'],
                   row['Precio_dto'], 
                   row['Precio'], 
                   row['Fecha'],
                   row['Url Imagen'], 
                   row['Url Producto'],
                   row['origen'],
                   row['tipo_vigente'],
                   row['pos'],
                   row['pagina_scraper'],
                   row['flag']) 
    cnxn.commit()
#cursor.close()
#cnxn.close()

In [31]:
#index


In [32]:
from sqlalchemy import create_engine
import urllib
params = urllib.parse.quote_plus('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+
                                 server+';DATABASE='+database+';UID='+username+';PWD='+ password)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [35]:
#pd.read_sql_table('sc2_detalle',engine)

In [36]:
#dfShein['id_sc2_corrida'] = int(maxCorrida[""][0])


In [37]:
#ACOMODAR EL DATAFRAME ACORDE A LOS CAMPOS DE LA BASE DE DATOS
#dfShein.rename(columns={'id Producto':'id_sc2_producto'},inplace=True)
#dfShein.rename(columns={'Talle':'talle'},inplace=True)
#dfShein.rename(columns={'Marca':'marca'},inplace=True)
#dfShein.rename(columns={'Tipo':'tipo'},inplace=True)
#dfShein.rename(columns={'Color':'color'},inplace=True)
#dfShein.rename(columns={'Sexo':'sexo'},inplace=True)
#dfShein.rename(columns={'Descripcion aux':'descripcion_aux'},inplace=True)
#dfShein.rename(columns={'Descripcion':'descripcion'},inplace=True)
#dfShein.rename(columns={'Stock':'stock'},inplace=True)
#dfShein.rename(columns={'Moneda':'moneda'},inplace=True)
#dfShein.rename(columns={'Precio':'precio'},inplace=True)
#dfShein.rename(columns={'Precio Original':'precio_original'},inplace=True)
#dfShein.rename(columns={'Fecha':'fecha_alta'},inplace=True)
#dfShein.rename(columns={'Url Imagen':'url_imagen'},inplace=True)
#dfShein.rename(columns={'Url Producto':'url_producto'},inplace=True)
#dfShein.rename(columns={'origen':'origen'},inplace=True)

In [38]:
#start=datetime.datetime.today()
#dfShein.to_sql(
#    name='sc2_detalle', # database table name
#    con=engine,
#    if_exists='append',
#    index=False)
#end=datetime.datetime.today()
#print(end-start)

In [33]:
#Arreglar tipo_vigente a tipo jb

cursor = cnxn.cursor()
cursor.execute("""
                
                update sc2_detalle set tipo_vigente= 'MUSCULOSA' 
                where tipo_vigente = 'REMERA & MUSCULOSA' and descripcion like '%tank%' 
                
                update sc2_detalle set tipo_vigente= 'REMERA' 
                where tipo_vigente = 'REMERA & MUSCULOSA' and descripcion not like '%tank%' 
                                
                update sc2_detalle set tipo_vigente= 'CHAQUETA' 
                where tipo_vigente = 'REMERA & MUSCULOSA'
                
                update sc2_detalle set tipo_vigente= 'SHORT' 
                where tipo_vigente = 'SHORT DE JEAN'
                
                update sc2_detalle set tipo_vigente= 'POLLERA' 
                where tipo_vigente = 'FALDA'
                                
                update sc2_detalle set tipo_vigente= 'CHOMBA' 
                where tipo_vigente = 'CAMISA DE POLO'
                
                update sc2_detalle set tipo_vigente= 'DUO' 
                where tipo_vigente = 'DUOS'
                                
                update sc2_detalle set tipo_vigente = 'CHAQUETA'
                where tipo_vigente = 'VAQUERO' and descripcion like '%Jacket%'

                update sc2_detalle set tipo_vigente = 'VESTIDO'
                where tipo_vigente ='VAQUERO' and descripcion like '%Dress%'

                update sc2_detalle set tipo_vigente = 'CAMISA' 
                where tipo_vigente ='VAQUERO' and descripcion like '%Shirt%' and descripcion not like '%Dress%'
                
                update sc2_detalle set tipo_vigente = 'ENTERITO' 
                where tipo_vigente ='VAQUERO' and descripcion like '%Jumpsuit%' 
                
                update sc2_detalle set tipo_vigente = 'ENTERITO' 
                where tipo_vigente ='VAQUERO' and descripcion like '%Overalls%'

                update sc2_detalle set tipo_vigente ='CHOMBA'
                where tipo_vigente = 'REMERA DE POLO' 
                
                update sc2_detalle set tipo_vigente ='PANTALON'
                where tipo_vigente = 'PANTALÓN' 
                
                update sc2_detalle set tipo_vigente ='CONJUNTO'
                where tipo_vigente = 'LENCERIA' and descripcion  like '%set%'

                update sc2_detalle set tipo_vigente ='CONJUNTO'
                where tipo_vigente = 'LENCERIA' and descripcion  not like '%set%' and descripcion like '%&%'

                update sc2_detalle set tipo_vigente ='BODY'
                where tipo_vigente = 'LENCERIA' and descripcion  not like '%set%' and descripcion like '%Dress%'
                
                update sc2_detalle set tipo_vigente ='CAMISON'
                where tipo_vigente = 'LENCERIA' and descripcion  not like '%set%' 
                and descripcion like '%Scallop%' and descripcion not like '%bodysuit%'

                update sc2_detalle set tipo_vigente ='BODY'
                where tipo_vigente = 'LENCERIA' and descripcion  not like '%set%' and descripcion like '%Bodysuit%'

                update sc2_detalle set tipo_vigente ='TRIANGULITO'
                where tipo_vigente = 'LENCERIA' and descripcion  not like '%set%' and descripcion like '%Triangle%'

                
                update sc2_detalle set tipo_vigente ='BIKINI'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion like '%BIKINI%'

                update sc2_detalle set tipo_vigente ='MALLA'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion like '%one piece%'

                update sc2_detalle set tipo_vigente ='MALLA'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion like '%one-piece%'

                update sc2_detalle set tipo_vigente ='SACO'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion not like '%one piece%' and descripcion like '%KIMONO%'

                update sc2_detalle set tipo_vigente ='ENTERITO'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion not like '%one piece%' and descripcion like '%KIMONO%'

                update sc2_detalle set tipo_vigente ='BIKINI'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion not like '%one piece%' and descripcion not like '%KIMONO%'  and descripcion like '%panty%'


                update sc2_detalle set tipo_vigente ='TOP'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion not like '%one piece%' 
                and descripcion not like '%KIMONO%'  and descripcion not like '%panty%' and descripcion like '%top%' and descripcion not like '%one-piece%' 

                update sc2_detalle set tipo_vigente ='BIKINI'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion not like '%one piece%' 
                and descripcion not like '%KIMONO%'  and descripcion not like '%panty%' and descripcion not like '%top%' and descripcion like '%MONOKINI%' 

                update sc2_detalle set tipo_vigente ='BIKINI'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion not like '%one piece%' 
                and descripcion not like '%KIMONO%'  and descripcion not like '%panty%' and descripcion not like '%top%' and descripcion not like '%MONOKINI%'  and descripcion not like '%one-piece%' and descripcion like '%set%'


                update sc2_detalle set tipo_vigente ='BIKINI'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%BIKINI%' and descripcion not like '%one piece%' 
                and descripcion not like '%KIMONO%'  and descripcion not like '%panty%' and descripcion not like '%top%' and descripcion not like '%MONOKINI%'  and descripcion not like '%one-piece%' 
                and descripcion not like '%set%' and descripcion like '%two piece%'
                
                update sc2_detalle set tipo_vigente ='PANTALON'
                where tipo_vigente ='PANTALON & JOGGIN' and descripcion like '%pants%'
                
                update sc2_detalle set tipo_vigente ='CALZA'
                where tipo_vigente ='PANTALON & JOGGIN' and descripcion not like '%pants%' and descripcion like '%leggings%'
                
                update sc2_detalle set tipo_vigente ='SHORT'
                where tipo_vigente ='PANTALON & JOGGIN' and descripcion not like '%pants%' 
                and descripcion not like '%leggings%' and descripcion like '%short%'

                update sc2_detalle set tipo_vigente ='JOGGING'
                where tipo_vigente ='PANTALON & JOGGIN' and descripcion not like '%pants%' 
                and descripcion not like '%leggings%' and descripcion not like '%short%' and descripcion like '%jogger%'

                update sc2_detalle set tipo_vigente ='PANTALON'
                where tipo_vigente ='PANTALÓN JOGGER' 
                
                update sc2_detalle set tipo_vigente ='BERMUDA'
                where tipo_vigente ='BERMUDA DE JEAN' 
                
                update sc2_detalle set tipo_vigente ='SHORT'
                where tipo_vigente ='TRAJE DE BAÑO' 
                
                update sc2_detalle set tipo_vigente ='BOXER ANATOMICO'
                where tipo_vigente = 'ROPA INTERIOR' and descripcion like '%trunks%'

                
                update sc2_detalle set tipo_vigente ='MUSCULOSA'
                where tipo_vigente = 'CAMISETAS S/MANGA' and descripcion like '%tank%'

                update sc2_detalle set tipo_vigente ='TOP'
                where tipo_vigente = 'CAMISETAS S/MANGA' and descripcion not like '%tank%' and descripcion like '%top%'

                update sc2_detalle set tipo_vigente ='MUSCULOSA'
                where tipo_vigente = 'CAMISETAS S/MANGA' and descripcion not like '%tank%' and descripcion not like '%top%' and descripcion like '%cami%'


                update sc2_detalle set tipo_vigente ='BLUSA'
                where tipo_vigente = 'CAMISETAS S/MANGA' and descripcion not like '%tank%' and descripcion not like '%top%' and descripcion not like '%cami%' and descripcion like '%Blouse%'

                update sc2_detalle set tipo_vigente ='TOP'
                where tipo_vigente = 'CAMISETAS S/MANGA' and descripcion not like '%tank%' and descripcion not like '%top%' and descripcion not like '%cami%' 
                and descripcion not like '%Blouse%' and descripcion like '%strip%'

                update sc2_detalle set tipo_vigente ='TOP'
                where tipo_vigente = 'CAMISETAS S/MANGA' and descripcion not like '%tank%' and descripcion not like '%top%' and descripcion not like '%cami%' 
                and descripcion not like '%Blouse%' and descripcion not like '%strip%'

                
                update sc2_detalle set tipo_vigente ='SACO'                
                where tipo_vigente = 'CARDIGAN' and descripcion like '%cardigan%'

                update sc2_detalle set tipo_vigente ='SWEATER'                
                where tipo_vigente = 'CARDIGAN' and descripcion not like '%cardigan%' and descripcion like '%sweater%'


                update sc2_detalle set tipo_vigente ='ENTERITO'                
                where tipo_vigente = 'CARDIGAN' and descripcion not like '%cardigan%' and descripcion not like '%sweater%' and descripcion like '%jumper%'


                update sc2_detalle set tipo_vigente ='SACO'                
                where tipo_vigente = 'CARDIGAN' and descripcion not like '%cardigan%' and descripcion not like '%sweater%' and descripcion not like '%jumper%' and descripcion like '%coat%'


                update sc2_detalle set tipo_vigente ='CHAQUETA'
                where tipo_vigente = 'CARDIGAN' and descripcion not like '%cardigan%' and descripcion not like '%sweater%' and descripcion not like '%jumper%' and descripcion not like '%coat%' and descripcion like '%jacket%'

                update sc2_detalle set tipo_vigente ='SACO'
                where tipo_vigente = 'CARDIGAN' and descripcion not like '%cardigan%' and descripcion not like '%sweater%' and descripcion not like '%jumper%' and descripcion not like '%coat%' and descripcion not like '%jacket%'

                update sc2_detalle set tipo_vigente ='CAMPERA'
                where tipo_vigente = 'CAMPERA BOMBER' 
                
                update sc2_detalle set tipo_vigente ='CAMISA'
                where tipo_vigente = 'CAMISA DE JEAN' 
                
                
                --ESTOS ULTIMO
                update sc2_detalle 
                set tipo_vigente = 'CHAQUETA'
                where tipo_vigente = 'CHAQUETA DE JEAN' and descripcion like '%jacket%' 

                update sc2_detalle 
                set tipo_vigente = 'SACO'
                where tipo_vigente = 'CHAQUETA DE JEAN' and descripcion not like '%jacket%' 
                
                update sc2_detalle 
                set tipo_vigente = 'DUOS'
                where tipo_vigente = 'DUO'
                                
                update sc2_detalle 
                set tipo_vigente = 'POLLERA'
                where tipo_vigente = 'VAQUERO' and 
                descripcion like '%skirt%'
                
                update sc2_detalle 
                set tipo_vigente = 'POLLERA'
                where tipo_vigente = 'VAQUERO' and descripcion not like '%skirt%' and descripcion like '%blouse%'


                update sc2_detalle 
                set tipo_vigente = 'ENTERITO'
                where tipo_vigente = 'VAQUERO' and descripcion not like '%skirt%'
                and descripcion not like '%blouse%' and descripcion like '%romper%'


                update sc2_detalle 
                set tipo_vigente = 'SHORT' 
                where tipo_vigente = 'VAQUERO' and descripcion not like '%skirt%' 
                and descripcion not like '%blouse%' 
                and descripcion not like '%romper%' and descripcion like'%short%'


                update sc2_detalle
                set tipo_vigente ='JEAN'
                where tipo_vigente = 'VAQUERO' and descripcion not like '%skirt%' 
                and descripcion not like '%blouse%' and descripcion not like '%romper%' 
                and descripcion not like'%short%'
                
                update sc2_detalle
                set tipo_vigente ='SACO'
                where tipo_vigente = 'TEJIDO' and descripcion like '%coat%'

                update sc2_detalle
                set tipo_vigente ='ENTERITO'
                where tipo_vigente = 'TEJIDO' and descripcion not like '%coat%' and descripcion like '%Jumpsuit%'

                update sc2_detalle
                set tipo_vigente ='SWEATER'
                where tipo_vigente = 'TEJIDO' and descripcion not like '%coat%' and descripcion not like '%Jumpsuit%'
                
                update sc2_detalle 
                set tipo_vigente = 'POLLERA'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion like '%skirt%'

                update sc2_detalle
                set tipo_vigente = 'MALLA'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' and descripcion like '%Swimsuit%'

                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' and descripcion not like '%Swimsuit%' and descripcion like '%dress%'

                update sc2_detalle
                set tipo_vigente ='MUSCULOSA'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion like '%tankini%'

                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' and descripcion like '%joyfunear%'


                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion like '%cover up%' and descripcion like '%front%'

                update sc2_detalle
                set tipo_vigente ='SACO'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion not like '%cover up%' and descripcion not like '%front%' and descripcion like '%coat%'


                update sc2_detalle
                set tipo_vigente ='POLLERA'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion not like '%cover up%' and descripcion not like '%front%' and descripcion not like '%coat%'
                and descripcion like '%skorts%'

                update sc2_detalle
                set tipo_vigente = 'TOP'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion not like '%cover up%' and descripcion not like '%front%' and descripcion not like '%coat%'
                and descripcion not like '%skorts%' and descripcion like '%bandeau%'


                update sc2_detalle 
                set tipo_vigente ='SHORT'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion not like '%cover up%' and descripcion not like '%front%' and descripcion not like '%coat%'
                and descripcion not like '%skorts%' and descripcion not like '%bandeau%' and descripcion like '%short%'



                update sc2_detalle 
                set tipo_vigente ='BIKINI'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion not like '%cover up%' and descripcion not like '%front%' and descripcion not like '%coat%'
                and descripcion not like '%skorts%' and descripcion not like '%bandeau%' and descripcion not like '%short%' and descripcion like '%Swimwear%'



                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion not like '%cover up%' and descripcion not like '%front%' and descripcion not like '%coat%'
                and descripcion not like '%skorts%' and descripcion not like '%bandeau%' and descripcion not like '%short%' and descripcion not like '%Swimwear%'
                and descripcion like '%crochet%'

                update sc2_detalle
                set tipo_vigente ='BIKINI'
                where tipo_vigente ='ROPA DE PLAYA' and descripcion not like '%skirt%' 
                and descripcion not like '%Swimsuit%' and descripcion not like '%dress%' and descripcion not like '%tankini%' 
                and descripcion not like '%joyfunear%' and descripcion not like '%cover up%' and descripcion not like '%front%' and descripcion not like '%coat%'
                and descripcion not like '%skorts%' and descripcion not like '%bandeau%' and descripcion not like '%short%' and descripcion not like '%Swimwear%'
                and descripcion not like '%crochet%'
                
                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente = 'ROPA DE PLAYA'

                update sc2_detalle
                set tipo_vigente ='SACO PIJAMA'
                where tipo_vigente = 'ROPA INTERIOR'
                and descripcion like'%belted%'


                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT'
                where tipo_vigente = 'ROPA INTERIOR'
                and descripcion not like'%belted%' and descripcion like '%shorts%' and descripcion not like'%&%'


                update sc2_detalle
                set tipo_vigente ='PIJAMA REMERA'
                where tipo_vigente = 'ROPA INTERIOR'
                and descripcion not like'%belted%' and descripcion not like '%shorts%' and descripcion not like'%&%' 

                update sc2_detalle
                set tipo_vigente ='PIJAMA REMERA'
                where tipo_vigente = 'ROPA INTERIOR'
                
                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente = 'DOS PIEZAS' and descripcion like '%dress%'
                
                
                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente = 'LENCERIA' and descripcion like '%slips%'
                
                update sc2_detalle
                set tipo_vigente = 'CONJUNTO'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion like '%with%'


                update sc2_detalle
                set tipo_vigente ='TOP'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion like'%bralette%'

                update sc2_detalle
                set tipo_vigente = 'ENTERITO'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion like '%romper%'

                update sc2_detalle
                set tipo_vigente ='BATA'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion like'%robe%'


                update sc2_detalle
                set tipo_vigente ='BOMBACHA'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion not like'%robe%' and descripcion like '%panty%'

                update sc2_detalle
                set tipo_vigente ='MUSCULOSA'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion not like'%robe%' and descripcion not like '%panty%'
                and descripcion like'%cami%'

                update sc2_detalle
                set tipo_vigente ='BODY'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion not like'%robe%' and descripcion not like '%panty%'
                and descripcion not like'%cami%' and descripcion like'%body%'

                update sc2_detalle
                set tipo_vigente ='TOP'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion not like'%robe%' and descripcion not like '%panty%'
                and descripcion not like'%cami%' and descripcion not like'%body%' and descripcion like '%top%'

                update sc2_detalle
                set tipo_vigente ='TANGA'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion not like'%robe%' and descripcion not like '%panty%'
                and descripcion not like'%cami%' and descripcion not like'%body%' and descripcion not like '%top%' and descripcion like'%thong%'


                update sc2_detalle
                set tipo_vigente ='BODY'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion not like'%robe%' and descripcion not like '%panty%'
                and descripcion not like'%cami%' and descripcion not like'%body%' and descripcion not like '%top%' and descripcion not like'%thong%'
                and descripcion like'%jumpsuit%'

                update sc2_detalle
                set tipo_vigente ='TOP'
                where tipo_vigente = 'LENCERIA' and descripcion not like '%slips%' and descripcion not like '%with%' and descripcion not like'%bralette%'
                and descripcion not like '%romper%' and descripcion not like'%robe%' and descripcion not like '%panty%'
                and descripcion not like'%cami%' and descripcion not like'%body%' and descripcion not like '%top%' and descripcion not like'%thong%'
                and descripcion not like'%jumpsuit%'

                
                update sc2_detalle
                set tipo_vigente ='BATA'
                where tipo_vigente ='PIJAMA'
                and descripcion like '%robe%'
                
                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT+PIJAMA REMERA'
                where tipo_vigente ='PIJAMA' and descripcion like'%top%' and descripcion like'%short%'


                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT+TOP'
                where tipo_vigente ='PIJAMA' and descripcion like'%top%' and descripcion like'%short%' and descripcion like'%bandeau%'

                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT+PIJAMA MUSCULOS'
                where tipo_vigente ='PIJAMA' and descripcion not like'%top%' and descripcion like'%short%' and descripcion not like'%bandeau%' 
                and descripcion like'%cami%'


                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT+TOP'
                where tipo_vigente ='PIJAMA' and descripcion not like'%top%' and descripcion not like'%bandeau%' 
                and descripcion not like'%cami%' and descripcion like'%short%' and descripcion like '%bra%'

                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT+PIJAMA REMERA'
                where tipo_vigente ='PIJAMA' and descripcion not like'%top%' and descripcion not like'%bandeau%' 
                and descripcion not like'%cami%' and descripcion not like '%bra%' and descripcion like'%short%' 
                and descripcion like '%tee%'


                update sc2_detalle 
                set tipo_vigente ='PIJAMA SHORT+CAMISA'
                where tipo_vigente ='PIJAMA' and descripcion not like'%top%' and descripcion not like'%bandeau%' 
                and descripcion not like'%cami%' and descripcion not like '%bra%' and descripcion like'%short%' 
                and descripcion not like '%tee%'


                update sc2_detalle
                set tipo_vigente ='VESTIDO'
                where tipo_vigente ='PIJAMA' and descripcion not like'%top%' and descripcion not like'%bandeau%' 
                and descripcion not like'%cami%' and descripcion not like '%bra%' and descripcion not like'%short%' 
                and descripcion not like '%tee%' and descripcion like '%dress%'
                
                
                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+PIJAMA MUSCULOS'
                where tipo_vigente ='PIJAMA' and descripcion like '%cami%' and descripcion like '%pants%'



                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+PIJAMA REMERA'
                where tipo_vigente ='PIJAMA' and descripcion not like '%cami%' and descripcion like '%pants%' 
                and descripcion like '%tee%'


                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+BLUSA'
                where tipo_vigente ='PIJAMA' and descripcion not like '%cami%' 
                and descripcion like '%pants%' and descripcion not like'%tee%'
                and descripcion like '%blouse%'


                update sc2_detalle
                set tipo_vigente = 'PANTALON PIJAMA+PIJAMA CAMISA'
                where tipo_vigente ='PIJAMA' and descripcion not like '%cami%' 
                and descripcion like '%pants%' and descripcion not like'%tee%'
                and descripcion not like '%blouse%' and descripcion like '%shirt%'
                
                update sc2_detalle
                set tipo_vigente ='POLLERA+PIJAMA MUSCULOS'
                where tipo_vigente ='PIJAMA' and descripcion like '%cami%' and descripcion like '%skirt%'

                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT+PIJAMA MUSCULOS'
                where tipo_vigente ='PIJAMA' and descripcion like '%cami%' and descripcion like'%Crisscross%'

                update sc2_detalle
                set tipo_vigente ='PIJAMA SHORT+PIJAMA MUSCULOS'
                where tipo_vigente ='PIJAMA' and descripcion not like'%Crisscross%' and descripcion like '%cami%' 

                
                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+TOP'
                where tipo_vigente ='PIJAMA' and descripcion like '%pants%'
                and descripcion like '%bra%'

                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+PIJAMA REMERA'
                where tipo_vigente ='PIJAMA' and descripcion like '%pants%'
                and descripcion not like '%bra%'

                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+PIJAMA REMERA'
                where tipo_vigente ='PIJAMA' and descripcion not like '%pants%'
                and descripcion not like '%bra%'
                and descripcion like '%satin%'

                
                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+PIJAMA CAMISA'
                where tipo_vigente ='PIJAMA' and descripcion not like '%pants%'
                and descripcion not like '%bra%'
                and descripcion not like '%satin%'
                and descripcion like '%button-up%'

                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+PIJAMA REMERA'
                where tipo_vigente ='PIJAMA' and descripcion not like '%pants%'
                and descripcion not like '%bra%'
                and descripcion not like '%satin%'
                and descripcion not like '%button-up%'
                and descripcion like '%plush%'

                update sc2_detalle
                set tipo_vigente='PIJAMA SHORT+TOP'
                where tipo_vigente ='PIJAMA' and descripcion not like '%pants%'
                and descripcion not like '%bra%'
                and descripcion not like '%satin%'
                and descripcion not like '%button-up%'
                and descripcion not like '%plush%'
                and descripcion like '%short%'

                update sc2_detalle
                set tipo_vigente ='PANTALON PIJAMA+PIJAMA REMERA'
                where tipo_vigente ='PIJAMA' and descripcion not like '%pants%'
                and descripcion not like '%bra%'
                and descripcion not like '%satin%'
                and descripcion not like '%button-up%'
                and descripcion not like '%plush%'
                and descripcion not like '%short%'

                update sc2_detalle
                set tipo_vigente ='MUSCULOSA+SHORT'
                where tipo_vigente ='DOS PIEZAS' and descripcion like '%shorts%' and descripcion like '%tank%'

                update sc2_detalle
                set tipo_vigente ='MUSCULOSA+SHORT' 
                where tipo_vigente ='DOS PIEZAS'
                and descripcion like '%shorts%' and descripcion not like '%tank%'
                and descripcion like '%cami top%'

                update sc2_detalle
                set tipo_vigente ='REMERA+SHORT'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion like '%shorts%' and descripcion not like '%tank%'
                and descripcion not like '%cami top%'

                update sc2_detalle
                set tipo_vigente ='REMERA+POLLERA'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%shorts%' and descripcion not like '%tank%'
                and descripcion not like '%cami top%'
                and descripcion like '%skirt%' and descripcion like '%tie%'

                update sc2_detalle
                set tipo_vigente ='REMERA+CALZA'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%shorts%' and descripcion not like '%tank%'
                and descripcion not like '%cami top%'
                and descripcion not like '%skirt%' and descripcion like '%tie%'
                and descripcion like '%legging%'

                update sc2_detalle
                set tipo_vigente ='BUZO+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%shorts%' and descripcion not like '%tank%'
                and descripcion not like '%cami top%'
                and descripcion not like '%skirt%' 
                and descripcion not like '%legging%' and descripcion like '%Sweatshirt%'
                and descripcion like '%pants%'

                update sc2_detalle
                set tipo_vigente ='BUZO+JOGGING'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%shorts%' and descripcion not like '%tank%'
                and descripcion not like '%cami top%'
                and descripcion not like '%skirt%' 
                and descripcion not like '%legging%' and descripcion like '%Sweatshirt%'
                and descripcion not like '%pants%'

                update sc2_detalle
                set tipo_vigente ='ENTERITO'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion like '%overalls%'

                update sc2_detalle
                set tipo_vigente='BUZO+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion like '%Sweatpants%'
                and descripcion like '%print%'

                update sc2_detalle
                set tipo_vigente ='BUZO+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion like '%Sweatpants%'
                and descripcion not like '%print%'
                
                update sc2_detalle
                set tipo_vigente ='ENTERITO'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion like '%jumpsuit%'

                update sc2_detalle
                set tipo_vigente ='REMERA + POLLERA'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion like '%skirt%'


                update sc2_detalle
                set tipo_vigente ='REMERA+CALZA'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion like '%legging%'


                update sc2_detalle
                set tipo_vigente ='BUZO+JOGGING'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion not like '%legging%'
                and descripcion like '%joggers%'

                update sc2_detalle
                set tipo_vigente ='MUSCULOSA+SHORT'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion not like '%legging%'
                and descripcion not like '%joggers%'
                and descripcion like '%short%'


                update sc2_detalle
                set tipo_vigente ='BLUSA+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion not like '%legging%'
                and descripcion not like '%joggers%'
                and descripcion not like '%short%'
                and descripcion like '%blouse%'



                update sc2_detalle
                set tipo_vigente ='MUSCULOSA+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion not like '%legging%'
                and descripcion not like '%joggers%'
                and descripcion not like '%short%'
                and descripcion not like '%blouse%'
                and descripcion like '%cami top%'

                
                update sc2_detalle
                set tipo_vigente ='MUSCULOSA+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion not like '%legging%'
                and descripcion not like '%joggers%'
                and descripcion not like '%short%'
                and descripcion not like '%blouse%'
                and descripcion not like '%cami top%'
                and descripcion like '%tank%'

                update sc2_detalle
                set tipo_vigente ='BUZO+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion not like '%legging%'
                and descripcion not like '%joggers%'
                and descripcion not like '%short%'
                and descripcion not like '%blouse%'
                and descripcion not like '%cami top%'
                and descripcion not like '%tank%'
                and descripcion like '%Hoodie%'


                update sc2_detalle
                set tipo_vigente ='REMERA+PANTALON'
                where tipo_vigente ='DOS PIEZAS' 
                and descripcion not like '%overalls%'
                and descripcion not like '%Sweatpants%'
                and descripcion not like '%print%'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%skirt%'
                and descripcion not like '%legging%'
                and descripcion not like '%joggers%'
                and descripcion not like '%short%'
                and descripcion not like '%blouse%'
                and descripcion not like '%cami top%'
                and descripcion not like '%tank%'
                and descripcion not like '%Hoodie%'
                

                update sc2_detalle
                set tipo_vigente ='ENTERITO'
                where tipo_vigente ='DOS PIEZAS'
                and descripcion like '%jumpsuit%'


                update sc2_detalle
                set tipo_vigente ='REMERA+PANTALON'
                where tipo_vigente ='DOS PIEZAS'
                and descripcion not like '%jumpsuit%'
                and descripcion like '%tee%'

                update sc2_detalle
                set tipo_vigente='BUZO+PANTALON'
                where tipo_vigente ='DOS PIEZAS'
                and descripcion not like '%jumpsuit%'
                and descripcion not like '%tee%'
                """)
cnxn.commit()
#cursor.close()
#cnxn.close()

In [34]:
#Arreglar +
cursor = cnxn.cursor()
cursor.execute("""



                update sc2_detalle set tipo_vigente= 'REMERA+POLLERA' 
                where tipo_vigente = 'REMERA + POLLERA' 
                                
                update sc2_detalle set tipo_vigente= 'PANTALON PIJAMA+TOP' 
                where tipo_vigente = 'PANTALON PIJAMA + TOP' 
                
                update sc2_detalle set tipo_vigente= 'PANTALON PIJAMA+PIJAMA CAMISA' 
                where tipo_vigente = 'PANTALON PIJAMA + PIJAMA CAMISA' 
                
                update sc2_detalle set tipo_vigente= 'PIJAMA SHORT+CAMISA' 
                where tipo_vigente = 'PIJAMA SHORT + CAMISA' 
                
                update sc2_detalle set tipo_vigente= 'PANTALON PIJAMA+PIJAMA REMERA' 
                where tipo_vigente = 'PANTALON PIJAMA + PIJAMA REMERA'
                
                update sc2_detalle set tipo_vigente= 'BUZO+JOGGING' 
                where tipo_vigente = 'BUZO + JOGGING' 
                
                update sc2_detalle set tipo_vigente= 'REMERA+SHORT' 
                where tipo_vigente = 'REMERA + SHORT' 
                
                update sc2_detalle set tipo_vigente= 'PANTALON PIJAMA+PIJAMA MUSCULOS' 
                where tipo_vigente = 'PANTALON PIJAMA + PIJAMA MUSCULOS' 
                
                update sc2_detalle set tipo_vigente= 'BUZO+PANTALON' 
                where tipo_vigente = 'BUZO + PANTALON' 
                
                update sc2_detalle set tipo_vigente= 'PIJAMA SHORT+PIJAMA REMERA' 
                where tipo_vigente = 'PIJAMA SHORT + PIJAMA REMERA' 
                
                update sc2_detalle set tipo_vigente= 'POLLERA+PIJAMA MUSCULOS' 
                where tipo_vigente = 'POLLERA + PIJAMA MUSCULOS' 
                
                update sc2_detalle set tipo_vigente= 'PANTALON PIJAMA+BLUSA' 
                where tipo_vigente = 'PANTALON PIJAMA + BLUSA' 
                
                update sc2_detalle set tipo_vigente= 'PIJAMA SHORT+TOP' 
                where tipo_vigente = 'PIJAMA SHORT + TOP' 
                
                update sc2_detalle set tipo_vigente= 'BLUSA+PANTALON' 
                where tipo_vigente = 'BLUSA + PANTALON' 
                
                
                update sc2_detalle set tipo_vigente= 'REMERA+CALZA' 
                where tipo_vigente = 'REMERA + CALZA' 
                
                update sc2_detalle set tipo_vigente= 'PIJAMA SHORT+PIJAMA MUSCULOS' 
                where tipo_vigente = 'PIJAMA SHORT + PIJAMA MUSCULOS' 
                
                update sc2_detalle set tipo_vigente= 'MUSCULOSA+SHORT' 
                where tipo_vigente = 'MUSCULOSA + SHORT' 
                
                update sc2_detalle set tipo_vigente= 'MUSCULOSA+PANTALON' 
                where tipo_vigente = 'MUSCULOSA + PANTALON'
                
                update sc2_detalle set tipo_vigente= 'REMERA+PANTALON' 
                where tipo_vigente = 'REMERA + PANTALON'
                
                """)
cnxn.commit()
#cursor.close()
#cnxn.close()

In [35]:
RESPALDO = pd.read_sql("select * from sc2_detalle where flag = 1 and origen ='SHEIN'",engine)

In [43]:
len(RESPALDO)
#2122431
#1957215
#1632829
#1225438
#1016189

2122431

In [36]:
g = RESPALDO.groupby(['id_sc2_producto','talle','color'])

In [37]:
REPE = 0
REPETIDOS = []
for i in g['tipo_vigente']:
    #i[1].unique() devuelve el tipo_vigente de cada producto,
    if len(i[1].unique()) != 1:
        REPETIDOS.append(i)
        REPE +=1

if REPE == 0:
    print('NO HAY','☺')
else:
    print(REPE,'☻')

#29
#125
#16
#28
#130 
#16
#12
#120
#235 
#883   


29 ☻


In [ ]:
╔═╦══
╠╦╬╦═
╚╩╩╩═

In [86]:
for item in REPETIDOS:
        
    print("""update sc2_detalle set tipo_vigente= '%s' where id_sc2_producto= '%s' and talle = '%s' and color = '%s' """%
          (item[1].iloc[-1],item[0][0],item[0][1],item[0][2]))
    break

update sc2_detalle set tipo_vigente= 'TOP' where id_sc2_producto= 'swblouse04190509401' and talle = 'L' and color = 'Multicolor' 


In [38]:
ex=[]
cursor = cnxn.cursor()
for item in REPETIDOS:
    try:
        cursor.execute("""update sc2_detalle set tipo_vigente= '%s' where id_sc2_producto= '%s' and talle = '%s' and color = '%s' """%(item[1].iloc[-1],item[0][0],item[0][1],item[0][2]))
        cnxn.commit()
    except:
        ex.append(item)
        print("EXCEPT: ",item[1].iloc[-1],item[0][0],item[0][1],item[0][2])
#cursor.close()
#cnxn.close()

In [63]:
# ex=[]
# updates = []
# for item in REPETIDOS:
#     try:
#         updates.append("""update sc2_detalle set tipo_vigente= '%s' where id_sc2_producto= '%s' and talle = '%s' and color = '%s' """%(item[1].iloc[-1],item[0][0],item[0][1],item[0][2]))
        
#     except:
#         ex.append(item)
#         print("EXCEPT: ",item[1].iloc[-1],item[0][0],item[0][1],item[0][2])
# #cursor.close()
# #cnxn.close()

In [62]:
"""update sc2_detalle set tipo_vigente= '%s' where id_sc2_producto= '%s' and talle = '%s' and color = '%s' """%(item[1].iloc[-1],item[0][0],item[0][1],item[0][2])

"update sc2_detalle set tipo_vigente= 'MUSCULOSA' where id_sc2_producto= 'vestmmc180622703' and talle = 'XS' and color = 'Red' "

In [65]:
updates

["update sc2_detalle set tipo_vigente= 'CAMPERA' where id_sc2_producto= 'jacket180822005' and talle = 'L' and color = 'Multicolor' ",
 "update sc2_detalle set tipo_vigente= 'CAMPERA' where id_sc2_producto= 'jacket180822005' and talle = 'M' and color = 'Multicolor' ",
 "update sc2_detalle set tipo_vigente= 'CAMPERA' where id_sc2_producto= 'jacket180822005' and talle = 'XL' and color = 'Multicolor' ",
 "update sc2_detalle set tipo_vigente= 'CAMPERA' where id_sc2_producto= 'jacket180822005' and talle = 'XXL' and color = 'Multicolor' ",
 "update sc2_detalle set tipo_vigente= 'CAMPERA' where id_sc2_producto= 'jacket180822005' and talle = 'XXXL' and color = 'Multicolor' ",
 "update sc2_detalle set tipo_vigente= 'CAMPERA' where id_sc2_producto= 'jacket180822005' and talle = 'XXXXL' and color = 'Multicolor' ",
 "update sc2_detalle set tipo_vigente= 'CHOMBA' where id_sc2_producto= 'skshirt07190814046' and talle = '10Y' and color = 'Multicolor' ",
 "update sc2_detalle set tipo_vigente= 'CHOMBA' 

In [52]:
server = 'tcp:192.168.1.6'
database ="estimaciones"
username = 'sa'
password = 'sa' 
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

In [54]:
tipologia_jb = 'select * from tipos'
tipologia_jb = pd.read_sql(tipologia_jb,cnxn)

In [56]:
tipologia_jb = tipologia_jb[['codigo','nombre']]

In [141]:
#tipologia_jb
#pd.DataFrame(RESPALDO['tipo_vigente'].unique())[pd.DataFrame(RESPALDO['tipo_vigente'].unique()).isin(tipologia_jb['nombre'])]


pd.DataFrame(RESPALDO['tipo_vigente'].unique())[~pd.DataFrame(RESPALDO['tipo_vigente'].unique())
                                                .set_index([0]).index.isin(tipologia_jb.set_index(['nombre']).index)]

,0
16,DOS PIEZAS
17,PANTALON PIJAMA + PIJAMA REMERA
18,PIJAMA SHORT + CAMISA
19,PANTALON PIJAMA + PIJAMA CAMISA
24,PIJAMA SHORT + TOP
25,PIJAMA SHORT+PIJAMA REMERA
26,PIJAMA SHORT + PIJAMA REMERA
35,PIJAMA SHORT + PIJAMA MUSCULOS
36,PANTALON PIJAMA + TOP
37,PANTALON PIJAMA + PIJAMA MUSCULOS


In [67]:
tipologia_jb[tipologia_jb['nombre']=='CHOMBA']

,codigo,nombre
23,30,CHOMBA


In [68]:
RESPALDO[RESPALDO['tipo_vigente']=='DUO'].head()

,id_sc2_producto,id_sc2_corrida,talle,marca,tipo,color,sexo,descripcion_aux,descripcion,stock,moneda,precio,precio_original,fecha_alta,url_imagen,url_producto,origen,tipo_vigente
45770,smtop03190329151,17,S,SHEIN,REMERA & MUSCULOSA,Black and White,Hombre,0,Men Asymmetric Hem Solid Basics Tee 2pcs,1,DOLAR US$,22.0,22.0,2019-09-23,https://img.ltwebstatic.com/images2_pi/2019/03...,https://www.shein.com/Men-Asymmetric-Hem-Solid...,SHEIN,DUO
45771,smtop03190329151,17,M,SHEIN,REMERA & MUSCULOSA,Black and White,Hombre,0,Men Asymmetric Hem Solid Basics Tee 2pcs,1,DOLAR US$,22.0,22.0,2019-09-23,https://img.ltwebstatic.com/images2_pi/2019/03...,https://www.shein.com/Men-Asymmetric-Hem-Solid...,SHEIN,DUO
45772,smtop03190329151,17,L,SHEIN,REMERA & MUSCULOSA,Black and White,Hombre,0,Men Asymmetric Hem Solid Basics Tee 2pcs,1,DOLAR US$,22.0,22.0,2019-09-23,https://img.ltwebstatic.com/images2_pi/2019/03...,https://www.shein.com/Men-Asymmetric-Hem-Solid...,SHEIN,DUO
45773,smtop03190329151,17,XL,SHEIN,REMERA & MUSCULOSA,Black and White,Hombre,0,Men Asymmetric Hem Solid Basics Tee 2pcs,1,DOLAR US$,22.0,22.0,2019-09-23,https://img.ltwebstatic.com/images2_pi/2019/03...,https://www.shein.com/Men-Asymmetric-Hem-Solid...,SHEIN,DUO
45774,smtop03190329151,17,XXL,SHEIN,REMERA & MUSCULOSA,Black and White,Hombre,0,Men Asymmetric Hem Solid Basics Tee 2pcs,1,DOLAR US$,22.0,22.0,2019-09-23,https://img.ltwebstatic.com/images2_pi/2019/03...,https://www.shein.com/Men-Asymmetric-Hem-Solid...,SHEIN,DUO


In [69]:
pd.DataFrame(RESPALDO['tipo_vigente'].unique())

,0
0,BLUSA
1,TOP
2,REMERA & MUSCULOSA
3,MUSCULOSA
4,JEAN
5,CHAQUETA DE JEAN
6,SHORT DE JEAN
7,CALZA
8,TOP DEPORTIVO
9,FALDA


In [96]:

numeros1 = [3,5,4,3,6,8]

numeros2 = [2,3,5,8,2,4]

In [117]:
numeros1.extend(numeros2)

In [128]:
sum([n1 for n1 in numeros1  if n1%3 == 0])

15

In [124]:
A

[3, 3, 6, 3]

In [108]:
import numpy as np


In [ ]:
np.dot(numeros1,numeros2)

In [116]:
[(n1+n2) for n1 in numeros1 for n2 in numeros2 if n2%n1 == 0]

[6, 10, 12, 8, 6, 16]